In [1]:
import numpy as np
import pandas as pd
import imblearn                                      
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

In [2]:
data = pd.read_csv('initial_preprocessing.csv')
data = data[data['3MResponse'] != 'Unknown']    ### We don't need unknown category for method-1


In [3]:
rs = 1337
X = data.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set
y = data['3MResponse']
model_label = LabelEncoder()
y = model_label.fit_transform(y.astype('str'))           ### encoding the target variable
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.1,random_state = rs)

<ipython-input-3-5503efe7e8e4>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = data.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set


In [4]:
imputer = KNNImputer(n_neighbors=15)                     ### Imputing the null values using KNN with neighbors as 15
fit_train = imputer.fit(X_train)
imput_train = fit_train.transform(X_train)
c = list(X_train.columns)
imput_train_df = pd.DataFrame(imput_train, columns = c)

In [5]:
imput_test = fit_train.transform(X_test)                 ### Imputing the test set using the training set
c = list(X_test.columns)
imput_test_df = pd.DataFrame(imput_test, columns = c)

In [9]:
labels = ['grp','gender','final_education','race_grp','newsmoker','drinker','ara_func_class']     ### normalizing the dataset

for i in imput_train_df.columns:
    if i not in labels:
        mean = imput_train_df[i].mean()
        std = imput_train_df[i].std()
        imput_train_df[i] = (imput_train_df[i] - mean) / std
        imput_test_df[i] = (imput_test_df[i] - mean) / std

In [10]:
labels = ['pres_imuran','pres_minocin','num_tnf','num_nontnf','hxunstab_ang','pres_minocin_BL','ethnicity','hxstroke','pres_imuran_BL']    #### This columns has same value for every row so when I divide by mean and std the value goes to infinity
for i in labels:
    imput_train_df = imput_train_df.drop(i,1)
    imput_test_df = imput_test_df.drop(i,1)

<ipython-input-10-649ae34e813e>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  imput_train_df = imput_train_df.drop(i,1)
<ipython-input-10-649ae34e813e>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  imput_test_df = imput_test_df.drop(i,1)


In [11]:
def sampling(over_under,X,y):                                        ### Over Sampling the dataset using Smote method
    if over_under == 'under':
        sampler = imblearn.under_sampling.RandomUnderSampler()
        X_under, y_under = sampler.fit_resample(X, y)
        return X_under,y_under
    elif over_under == 'over':
        ros = RandomOverSampler(random_state=rs)
        X_over, y_over = ros.fit_resample(X, y)
        return X_over,y_over
    else:
        sampler = SMOTE(random_state = rs)
        X_smote, y_smote = sampler.fit_resample(X, y)
        return X_smote,y_smote
X_train_final,y_train_final = sampling('smote',imput_train_df,Y_train)

In [66]:
X_train_final['y'] = y_train_final
X_train_final.to_csv('train_method1_cf.csv', index = False)

In [67]:
imput_test_df['y'] = Y_test 
imput_test_df.to_csv('test_method1_cf.csv', index = False)